<a href="https://colab.research.google.com/github/Witcape/Emotion_Recognition_in_Live_Video_Data/blob/main/Emotion_Recognition_in_Video_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### STEP 1 - Import all the Libraries and google drive

In [1]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import sklearn
import tensorflow as tf
import pandas as pd
import sklearn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.activations import linear,relu,sigmoid
import cv2
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
loss = history.history['loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: ignored

## Step 2 - Loading Data & Preprocessing

In [3]:
train_data_dir = '/content/gdrive/MyDrive/Human emotions/train'
test_data_dir = '/content/gdrive/MyDrive/Human emotions/test'

In [4]:
train_data = tf.keras.utils.image_dataset_from_directory(train_data_dir)
test_data = tf.keras.utils.image_dataset_from_directory(test_data_dir)

Found 29031 files belonging to 7 classes.
Found 7378 files belonging to 7 classes.


In [5]:
train_data_iterator = train_data.as_numpy_iterator()
test_data_iterator = test_data.as_numpy_iterator()

In [6]:
train_batch = train_data_iterator.next()
test_batch = test_data_iterator.next()

In [7]:
test_batch

(array([[[[113., 113., 113.],
          [113., 113., 113.],
          [113., 113., 113.],
          ...,
          [ 96.,  96.,  96.],
          [ 96.,  96.,  96.],
          [ 96.,  96.,  96.]],
 
         [[113., 113., 113.],
          [113., 113., 113.],
          [113., 113., 113.],
          ...,
          [ 96.,  96.,  96.],
          [ 96.,  96.,  96.],
          [ 96.,  96.,  96.]],
 
         [[113., 113., 113.],
          [113., 113., 113.],
          [113., 113., 113.],
          ...,
          [ 96.,  96.,  96.],
          [ 96.,  96.,  96.],
          [ 96.,  96.,  96.]],
 
         ...,
 
         [[ 90.,  90.,  90.],
          [ 90.,  90.,  90.],
          [ 90.,  90.,  90.],
          ...,
          [ 76.,  76.,  76.],
          [ 76.,  76.,  76.],
          [ 76.,  76.,  76.]],
 
         [[ 90.,  90.,  90.],
          [ 90.,  90.,  90.],
          [ 90.,  90.,  90.],
          ...,
          [ 76.,  76.,  76.],
          [ 76.,  76.,  76.],
          [ 76.,  76.,  76.

In [8]:
len(test_batch)

2

In [9]:
train_batch[0].shape

(32, 256, 256, 3)

In [10]:
train_data = train_data.map(lambda x,y: (x/255,y))
test_data = test_data.map(lambda x,y: (x/255,y))

In [11]:
scaled_train_iterator = train_data.as_numpy_iterator()
scaled_test_iterator = test_data.as_numpy_iterator()
train_batch = scaled_train_iterator.next()
test_batch = scaled_test_iterator.next()
test_batch[0].max()

1.0

In [12]:
len(train_data)

908

In [13]:
train_size = int(len(train_data)*0.7)
val_size = int(len(train_data)*0.3)+1
test_size = len(test_data)
train_size+val_size

908

In [14]:
train = train_data.take(train_size)
val = train_data.skip(train_size).take(val_size)
test = test_data.take(test_size)

In [15]:
print(len(train),len(val),len(test))

635 273 231


## Step 3 - Model

In [16]:
model = Sequential(
    [
        Conv2D(16, (3,3), 1,  activation = 'relu', input_shape = (256, 256, 3)),
        MaxPooling2D(),

        Conv2D(32, (3,3), 1,  activation = 'relu'),
        MaxPooling2D(),

        Conv2D(16, (3,3), 1,  activation = 'relu', input_shape = (256, 256, 3)),
        MaxPooling2D(),

        Flatten(),

        Dense(256, activation = 'relu'),
        Dense(7, activation = 'softmax')
    ]
)

In [17]:
model.compile(
    optimizer = Adam(learning_rate = 0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [18]:
early_stopping = EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=3, monitor='val_loss', min_lr=0.00001)

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 16)       0

In [20]:
log_dir = '/content/gdrive/MyDrive/Human emotions/Logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)

In [24]:
history = model.fit(train, validation_data = val, epochs = 20, callbacks = [tensorboard_callback, early_stopping, reduce_lr])

Epoch 1/20
635/635 [==============================] - 97s 153ms/step - loss: 1.0318 - accuracy: 0.6190 - val_loss: 1.5111 - val_accuracy: 0.4586 - lr: 1.0000e-05
Epoch 2/20
635/635 [==============================] - 96s 151ms/step - loss: 1.0218 - accuracy: 0.6249 - val_loss: 1.5133 - val_accuracy: 0.4601 - lr: 1.0000e-05
Epoch 3/20
635/635 [==============================] - 97s 152ms/step - loss: 1.0155 - accuracy: 0.6281 - val_loss: 1.5167 - val_accuracy: 0.4603 - lr: 1.0000e-05
Epoch 4/20
635/635 [==============================] - 96s 151ms/step - loss: 1.0097 - accuracy: 0.6306 - val_loss: 1.5161 - val_accuracy: 0.4621 - lr: 1.0000e-05
Epoch 5/20
635/635 [==============================] - 97s 152ms/step - loss: 1.0010 - accuracy: 0.6332 - val_loss: 1.5152 - val_accuracy: 0.4636 - lr: 1.0000e-05
Epoch 6/20
635/635 [==============================] - 129s 203ms/step - loss: 0.9962 - accuracy: 0.6343 - val_loss: 1.5191 - val_accuracy: 0.4633 - lr: 1.0000e-05


In [25]:
model.save('/content/gdrive/MyDrive/Human emotions/Saved_model')
